In [1]:
!uname -a

Linux 6561fb1a6fcb 6.1.85+ #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux


In [2]:
!pip install Cython==0.29.36
!pip uninstall scipy -y
!pip install scipy==1.9
!pip uninstall pyparsing -y
!pip install pyparsing==2.4
!pip uninstall scikit_learn -y

Found existing installation: scipy 1.9.0
Uninstalling scipy-1.9.0:
  Successfully uninstalled scipy-1.9.0
  Using cached scipy-1.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.2 kB)
Using cached scipy-1.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-sklearn 0.15.0 requires scikit-learn<0.25.0,>=0.24.0, which is not installed.
fastai 2.7.16 requires scikit-learn, which is not installed.
librosa 0.10.2.post1 requires scikit-learn>=0.20.0, which is not installed.
sklearn-pandas 2.2.0 requires scikit-learn>=0.23.0, which is not installed.
smac 1.2 requires scikit-learn>=0.22.0, which is not installed.
xgboost 2.1.1 requires nvidia-nccl-cu12; platform_system == "Linux" and platform_machine != "aarch64", which is not installed.
albumentations 1.4.13 requires sci

In [1]:
!pip uninstall imbalanced-learn -y
!pip uninstall mlxtend -y
!pip uninstall yellowbrick -y

In [2]:
!pip install scikit-learn==0.24.2 --no-build-isolation

  Using cached scikit_learn-0.24.2-cp310-cp310-linux_x86_64.whl
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.13.0 requires scikit-learn>=1.2.2, but you have scikit-learn 0.24.2 which is incompatible.


In [3]:
!pip install auto-sklearn

In [4]:
import autosklearn.regression
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
df_train = pd.read_csv("TrainSet_regression.csv")
df_test = pd.read_csv("TestSet_regression.csv")

# features and target
X_train = df_train.drop(['Price'], axis=1)
y_train = df_train['Price']

X_test = df_test.drop(['Price'], axis=1)
y_test = df_test['Price']

#4h for training, 20 mins per model

automl_regressor = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task= 14400,
    per_run_time_limit= 1200
)

automl_regressor.fit(X_train, y_train, dataset_name="housing_prices")


print(automl_regressor.leaderboard())



[WARNING] [2024-08-18 17:58:40,341:Client-AutoML(1):housing_prices] Capping the per_run_time_limit to 59.0 to have time for a least 2 models in each process.
[WARNING] [2024-08-18 17:59:43,357:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-08-18 17:59:45,631:Client-EnsembleBuilder] No runs were available to build an ensemble from
          rank  ensemble_weight               type      cost   duration
model_id                                                               
4            1              1.0  gradient_boosting  0.751188  11.916081


In [7]:
import sklearn.metrics as skm

y_pred = automl_regressor.predict(X_test)

print(f"R2:                         {skm.r2_score(y_test, y_pred)}")
print(f"Mean Squared Error:         {skm.mean_squared_error(y_test, y_pred)}")
print(f"Root Mean Squared Error:    {skm.mean_squared_error(y_test, y_pred, squared=False)}")
print(f"Mean Absolute Error:        {skm.mean_absolute_error(y_test, y_pred)}")
print(f"Explained Variance:         {skm.explained_variance_score(y_test, y_pred)}")
print(f"Max Error:                  {skm.max_error(y_test, y_pred)}")
print(f"Median Absolute Error       {skm.median_absolute_error(y_test, y_pred)}")


R2:                         0.2702618020176065
Mean Squared Error:         2915821205598.2637
Root Mean Squared Error:    1707577.584064122
Mean Absolute Error:        1298851.009855313
Explained Variance:         0.27189978646138446
Max Error:                  7067876.0
Median Absolute Error       1021068.25


In [9]:
from google.colab import files
#export leaderboard

#leaderboard to df
leaderboard_df = automl_regressor.leaderboard(detailed=True)
leaderboard_df = pd.DataFrame(leaderboard_df)

print(leaderboard_df)

# download leaderboard
leaderboard_df.to_csv('autosklearn_regression_preprosessing_leaderboard.csv', index=False)

files.download('autosklearn_regression_preprosessing_leaderboard.csv')


          rank  ensemble_weight               type      cost   duration  \
model_id                                                                  
4            1              1.0  gradient_boosting  0.751188  11.916081   

          config_id  train_loss  seed    start_time      end_time  budget  \
model_id                                                                    
4                 3    0.297111     0  1.724004e+09  1.724004e+09     0.0   

                      status data_preprocessors      feature_preprocessors  \
model_id                                                                     
4         StatusType.SUCCESS                 []  [select_rates_regression]   

         balancing_strategy   config_origin  
model_id                                     
4                      None  Initial design  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# output of values to dictionary
metrics_dict = {
    'Metric': [
        'R2',
        'Mean Squared Error',
        'Root Mean Squared Error',
        'Mean Absolute Error',
        'Explained Variance',
        'Max Error',
        'Median Absolute Error'
    ],
    'Value': [
        skm.r2_score(y_test, y_pred),
        skm.mean_squared_error(y_test, y_pred),
        skm.mean_squared_error(y_test, y_pred, squared=False),
        skm.mean_absolute_error(y_test, y_pred),
        skm.explained_variance_score(y_test, y_pred),
        skm.max_error(y_test, y_pred),
        skm.median_absolute_error(y_test, y_pred)
    ]
}

#to df
metrics_df = pd.DataFrame(metrics_dict)


print(metrics_df)

metrics_df.to_csv('autosklearn_regression_preprosessing_metrics.csv', index=False)
files.download('autosklearn_regression_preprosessing_metrics.csv')

                    Metric         Value
0                       R2  2.702618e-01
1       Mean Squared Error  2.915821e+12
2  Root Mean Squared Error  1.707578e+06
3      Mean Absolute Error  1.298851e+06
4       Explained Variance  2.718998e-01
5                Max Error  7.067876e+06
6    Median Absolute Error  1.021068e+06


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>